# **Distributed graph embedding from cosmological simulations**

**Group 13**:

|student|email|ID|
|---|---|---|
|Lorenzo Cavezza|lorenzo.cavezza@studenti.unipd.it|2130648|
|Giulia Doda|giulia.doda@studenti.unipd.it|2104267|
|Giacomo Longaroni|giacomo.longaroni@studenti.unipd.it|2126898|
|Laura Ravagnani|laura.ravagnani@studenti.unipd.it|2104271|

### Introduction

Estimating cosmological parameters from the analysis of cosmic web structures is a key goal in cosmology since it helps us to uncover fundamental properties of the universe. The cosmic web is the large-scale structure of the universe, composed by galaxy clusters and filaments, and some important cosmological parameters from the $\Lambda \text{CDM}$ model (the most trusted model nowadays) are for example $\Omega_{m}$, which is the total matter density, and $\sigma_{8}$, which measures matter density fluctuations on a scale of $8\, \text{Mpc}$ (recalling that $1 \,\text{pc} \sim 3.26 \,\text{light years} \sim 31 \,\text{x}\, 10^{16} \,\text{m}$). 
<div class="container" style="display: inline-block;">
  <figure>
    <div style="padding: 10px; text-align: center;">
      <img src='cosmicweb.png' style="width: 450px; height: 250px;" />
      <figcaption><br>Fig. 1: the cosmic web.</b></figcaption>
    </div>
  </figure>
</div>

These parameters are traditionally estimated using summary statistics, such as the power spectrum. However, recent advances in machine learning have opened up a promising new approach. For example, **graph neural networks** (GNN) offers many advantages specifically for this task with respect to other types of neural networks (NN). Let’s first recall that a graph consists of a set of nodes connected by edges, that here we consider as symmetrical links between nodes (without a preferred direction). Both nodes and edges can be associated with data. Moreover, we can associate global features with the graph that are related to the entire graph rather than to nodes or edges. A GNN is then a NN that takes graphs as input and can make predictions about node or edge properties, or properties of the graph as a whole. This type of network offers many advantages for the cosmology task.  First of all, it is inherently permutation invariant, and we can design it to respect physical symmetries relevant to our task, such as rotation and translation invariance, so that the network can focus on learning relevant correlations. Another advantage is that, unlike some other network models like convolutional NN (CNN), GNN do not impose any cut in scale.  In CNN, the grid size determines the minimum scale, requiring a cutoff because CNN cannot handle arbitrarily fine grids due to memory limitations. In particular, when dealing with nonlinear scales, CNN need very fine grids, which are not feasible with current computational architectures. Finally the cosmic web can be easily represented as a graph and processing it with a GNN is more appropriate since a graph neural network handle sparse and irregular data, such as the cosmic web, more effectively.

Our goal is then to embed the cosmic web from simulated data into a graph so that it can be further processed by a GNN.
<div class="container" style="display: inline-block;"> 
    <figure>
      <div style="padding: 10px; text-align: center;">
        <img src='graph_8_997.jpeg' style="width: 450px; height: 450px;">
        <figcaption><br>Fig. 2: example of a cosmic graph from a simulation represented in 3D space.</b></figcaption>
      </div>
    </figure>
</div>

#### The data
Quijote is a suite of N-body simulation containing many different catalogues. We focus specifically on the halo catalogue at redshift $z=1$. This provides us with $2000$ dark matter halo simulations, where the halos are already identified by the friends of friends (FoF) algorithm. Each simulations contains $512^3$ particles in a comoving volume of $1\,\text{Gpc}$. The simulations are generated through latin hypercube and each of them is associated with different values for the cosmological parameters. 

#### The method
We need to embed each simulation into a graph. First, we find all halo pairs within a distance equal to or less than the linking radius, which we set at $0.2\,\text{Gpc} $, following previous studies. Next, we extract the translation and rotation invariant features: the distance between connected halos and two angles that specify the position of each linked halo pair relative to the halos' centroid. It's important to note that all distance calculations apply periodic boundary conditions, to account for the limitations of the simulation box.
<div class=\"container\" style=\"display: inline-block;\"> 
    <figure>
      <div style="padding: 10px; text-align: center;">
        <img src='graph_embedding_space.png' style="width: 350px; height: 250px;">
        <figcaption><br>Fig. 3: graph embedding space.</b></figcaption>
      </div>
    </figure>
</div>

Here we can easily visualize the graph embedding. Each node represents a halo, with its corresponding feature being the halo mass. The graph edges must account for gravitational interactions between halos and respect the required symmetries. Graph edges are defined by: 
- the linking distance $d$ between linked halos normalized by the linking radius, 
- the cosines of the two angles describing the position of the linked halo pair relative to the centroid of the halos. 

Finally, we also assign a global feature to each graph $u = log_{10}N$ where $N$ is the total number of halos in the simulation to summarize the cardinality of the graph.

#### Resources and cluster (TO DO BETTER)
- 4 virtual machines on CloudVeneto (4 VCPUs, 8GB RAM, 25GB storage)
  - VM1: 
    - hosting the Spark Master and a Spark worker
    - with a 250 GB volume mounted on
    - network file system (NFS) server 
  - VM2 $\rightarrow$ VM4: 
    - Spark workers
    - NFS clients

In [ ]:
import readfof
from pyspark.sql import SparkSession
import numpy as np
import matplotlib.pyplot as plt
import math

In [ ]:
spark = SparkSession.builder \
        .master("spark://master:7077")\
        .appName("CosmoSparkApplication")\
        .getOrCreate()

sc = spark.sparkContext

#### Functions

In [ ]:
# Read data
def read_cosmo_data(file_path):

    # Read Fof
    FoF = readfof.FoF_catalog(
        file_path,           # simulation directory
        2,                   # snapnum, indicating the redshift (z=1)
        long_ids = False,
        swap = False,
        SFR = False,
        read_IDs = False
        )

    return FoF



# Get masses and positions from FoF
def get_pos_mass(FoF):

    pos = FoF.GroupPos/1e06             # Halo positions in Gpc/h 
    mass_raw = FoF.GroupMass * 1e10     # Halo masses in Msun/h

    dim = pos.shape[0]
    pos_mass_matrix = np.hstack([pos, mass_raw.reshape(dim, 1)])

    return pos_mass_matrix



# To assign simulation keys to each point in each simulation
def assign_key_to_rows(key_value_pair):

    key, array = key_value_pair

    return [(key, row) for row in array]



# To assign each value of the rdd to each bin
def bin_placer(value, edges):

    for i in range(1, len(edges)):
        if value < edges[i]:
            return edges[i - 1]
        
    return edges[-1]



# mass cuts distribution histogram
def create_mass_hist(mass_rdd, n_sims):

    # bin edges 
    min_mass_cuts = mass_rdd.map(lambda x: x[1]).reduce(min)
    max_mass_cuts = mass_rdd.map(lambda x: x[1]).reduce(max)

    # number of bins
    num_bins = int(1 + np.log2(n_sims)) # Sturges's rule

    # bin width
    bin_width = (max_mass_cuts - min_mass_cuts) / num_bins

    # edges
    edges = [min_mass_cuts + i * bin_width for i in range(int(num_bins) + 1)]

    # histogram with map - reduce
    mapped_mass_cuts = mass_rdd.map(lambda x: (bin_placer(x[1], edges), 1))
    reduced_mass_cuts = mapped_mass_cuts.reduceByKey(lambda x, y: x + y)
    mass_cuts_distr = np.array(reduced_mass_cuts.collect())

    # create figure
    fig, ax = plt.subplots(figsize=(10,6))

    ax.hist(mass_cuts_distr[:, 0], bins=11, weights=mass_cuts_distr[:, 1], color='#0f5e6f', alpha=0.5, label='Mass distribution')
    ax.set_xlabel('Halo mass ($M_{\\odot}$)', fontsize=14)
    modot = '$M_{\\odot}$'
    b_width = "{:.2e}".format(bin_width)
    ax.set_ylabel(f'Counts  /  {b_width} ' + modot, fontsize=14)
    ax.grid(alpha=0.4, linestyle='--')



# number of halos distribution histogram
def create_halos_hist(halos_rdd, n_sims):
    
    # bin edges 
    min_n_halos = halos_rdd.map(lambda x: x).reduce(min)
    max_n_halos = halos_rdd.map(lambda x: x).reduce(max)

    # number of bins
    num_bins_halos = int(1 + np.log2(n_sims)) # Sturges's rule

    # bin width
    bin_width_halos = (max_n_halos - min_n_halos) / num_bins_halos

    # edges
    edges_halos = [min_n_halos + i * bin_width_halos for i in range(num_bins_halos + 1)]

    # histogram with map - reduce
    mapped_n_halos = halos_rdd.map(lambda x: (bin_placer(x, edges_halos), 1))
    reduced_n_halos = mapped_n_halos.reduceByKey(lambda x, y: x + y)
    n_halos_distr = np.array(reduced_n_halos.collect())

    # create histogram
    fig1, ax1 = plt.subplots(figsize=(10,6))

    ax1.hist(n_halos_distr[:, 0], bins=num_bins_halos, weights=n_halos_distr[:, 1], color='#f28a44', alpha=0.5, label='Mass distribution')
    ax1.set_xlabel('# of halos', fontsize=14)
    b_width = "{:.2e}".format(bin_width_halos)
    ax1.set_ylabel(f'Counts  /  {b_width} ', fontsize=14)
    ax1.grid(alpha=0.4, linestyle='--')



# create all the possible halo pairs inside each box
def create_pairs(pos_rdd, divisions):

    # indexed positions rdd (point indexes)
    # --> ( simkey, (point_idx, array(x, y, z)) )
    idx_pos_rdd = pos_rdd.groupByKey()\
                         .flatMapValues(lambda vals: enumerate(vals))

    # initialize boxes
    boxes = sub_box_bounds(divisions, 0.2)

    # indexed positions rdd with box assigned
    # --> ( simkey_boxkey, (point_idx, array(x, y, z)) )
    idx_pos_box_rdd = idx_pos_rdd.flatMapValues(lambda p: assign_box(p, boxes))\
                                 .map(lambda x: (str(x[0]) + '_' + x[1][0], x[1][1]))

    # obtain all the possible point pairs for each simulation clustered by boxes
    # --> ( (simkey_boxkey, (idx, array)), (simkey_boxkey, (idx, array)) )
    cartesian_rdd = idx_pos_box_rdd.groupByKey()\
                            .flatMapValues(lambda points: [(p1,p2) for p1 in points for p2 in points])\
                            .map(lambda x: ((x[0], x[1][0]),(x[0], x[1][1])))

    # compute differences between every pair 
    # --> (simkey_boxkey, (idx1, idx2, coord1, coord2, diff_coord))
    diff_rdd = cartesian_rdd.map(lambda x:(x[0][0],(x[0][1][0], x[1][1][0], x[0][1][1],  x[1][1][1] , x[0][1][1] - x[1][1][1])))

    # --> (simkey_boxkey, (idx1, idx2, coord1, coord2, diff_coord, norm))
    pairs_dist_rdd_with_box = diff_rdd.mapValues(lambda x: (x[0], x[1], x[2], x[3], x[4], np.linalg.norm(x[4])))

    pairs_dist_rdd = pairs_dist_rdd_with_box.map(lambda x: (int(x[0].split('_')[0]), (x[1])))\
                                                   .map(convert_to_tuple)\
                                                   .distinct()\
                                                   .map(convert_to_array)
    
    return pairs_dist_rdd



# Plot a graph in 3D space
def plot_graph_3D(num, pars_file, pos, masses, edge_idx):

    fig = plt.figure(figsize=(10, 10))
    fontsize = 12

    ax = fig.add_subplot(projection="3d")

    pos = np.array(pos, dtype=float) * 1.e3   # show in Mpc

    # Draw lines for each edge
    for (src, dst) in edge_idx:

        src = pos[int(src)].tolist()
        dst = pos[int(dst)].tolist()

        ax.plot([src[0], dst[0]], [src[1], dst[1]], zs=[src[2], dst[2]], linewidth=0.6, color='dimgrey')

    # Plot nodes
    mass_mean = np.mean(masses)
    for i,m in enumerate(masses):
            ax.scatter(pos[i, 0], pos[i, 1], pos[i, 2], s=50*m*m/(mass_mean**2), zorder=1000, alpha=0.6, color='mediumpurple')

    ax.xaxis.set_tick_params(labelsize=fontsize)
    ax.yaxis.set_tick_params(labelsize=fontsize)
    ax.zaxis.set_tick_params(labelsize=fontsize)

    ax.set_xlabel('x (Mpc)', fontsize=16, labelpad=15)
    ax.set_ylabel('y (Mpc)', fontsize=16, labelpad=15)
    ax.set_zlabel('z (Mpc)', fontsize=16, labelpad=15)

    rl = '$R_{link} = 0.2$'

    pars_file = pars_file[num]

    ax.set_title(f'\tGraph n°{num}, Masses $\\geq 99.7$% percentile, {rl} Mpc \t \n \n $\\Omega_m = {float(pars_file[0]):.3f}$ \t $\\sigma_8 = {float(pars_file[1]):.3f}$', fontsize=20)

    plt.show()



def get_edge_attributes(pos_rdd, linked_pairs_dist_rdd):
    # centroids positions
    halo_centroids = pos_rdd.reduceByKey(lambda x, y: (x + y) / 2)

    # joined rdd with halo centroids positions
    joined_rdd = linked_pairs_dist_rdd.join(halo_centroids)

    # distance between each point from each pair and halo centroid
    row_col_diff_rdd = joined_rdd.mapValues(
        lambda x: (
            x[0][0],        # idx_i
            x[0][1],        # idx_j
            x[0][2] - x[1], # row
            x[0][3] - x[1], # col
            x[0][4],        # diff
            x[0][5]         # dist
            ))

    # normalizing 
    normalized_rdd = row_col_diff_rdd.mapValues(
        lambda x: (
            x[0],                      # idx_i
            x[1],                      # idx_j
            x[2]/np.linalg.norm(x[2]), # row_normalized
            x[3]/np.linalg.norm(x[3]), # col_normalized
            x[4]/np.linalg.norm(x[4]), # s_ij
            x[5]/0.2                   # |d_ij|/r 
        )
    )

    # edge attributes
    edge_attr_rdd = normalized_rdd.mapValues(
        lambda x: (
            x[0],
            x[1],
            np.dot( x[2].T, x[3] ), # cos(alpha)
            np.dot( x[2].T, x[4] ), # cos(beta)
            x[5]                    # |d_ij|/r 
        )
    )

    return edge_attr_rdd



# Graph object
class graph:

    def __init__(self, node_f, pos, sim_pars, glob_f, edge_idx, edge_f):
        
        self.node_f = node_f
        self.pos = pos
        self.sim_pars = sim_pars
        self.glob_f = glob_f
        self.edge_idx = edge_idx
        self.edge_f = edge_f



# Create graph object
def create_graph(element):

    sim_graph = graph(
        np.array(element[0])[:,3],   # node_f = masses
        np.array(element[0])[:,0:3], # pos
        np.array(element[3]),        # sim_pars
        np.array(element[2]),        # glob_f
        np.array(element[1])[:,0:2], # edge_idx
        np.array(element[1])[:,2:5], # edge_f
    )

    return(sim_graph)



# Function that returns the partitions bounds as a dictionary of lists of tuples, 
# each tuple being the min and max of a dimension
def sub_box_bounds(box_number, r_link):

    sub_length = 1.0 / box_number # partition length
    bounds = {}
    base = 'box'
    sub_box_counter = 1

    for x in range(0, box_number):
        for y in range(0, box_number):
            for z in range(0, box_number):
                key = base + str(sub_box_counter)
                single_bounds = []
                centre = [x, y, z] # vertex of a sub_box corresponding to min x,y,z
                for i in range(3):
                    min_bound = round(max(0, centre[i] * sub_length - 0.5 * r_link), 2)
                    max_bound = round(min(1, centre[i] * sub_length + sub_length + 0.5 * r_link), 2)
                    single_bounds.append((min_bound, max_bound))
                bounds[key] = single_bounds
                sub_box_counter += 1

    return bounds



# Assign each point to a box
def assign_box(point, boxes):

    position = point[1]
    x, y, z = position
    box_assign = []
    
    for box_name, ((x_min, x_max), (y_min, y_max), (z_min, z_max)) in boxes.items():
     if (x_min <= x <= x_max) and (y_min <= y <= y_max) and (z_min <= z <= z_max):
           box_assign.append((box_name, point))
    
    return box_assign



# Convert all element of an rdd into a tuple
def convert_to_tuple(data):

    return (
        data[0],
        data[1][0],
        data[1][1],
        (float(data[1][2][0]), float(data[1][2][1]), float(data[1][2][2])),  # from array to tuple
        (float(data[1][3][0]), float(data[1][3][1]), float(data[1][3][2])),  # from array to tuple
        (float(data[1][4][0]), float(data[1][4][1]), float(data[1][4][2])),  # from array to tuple
        float(data[1][5])             
    )



# Convert vectors of an rdd into a np.array (key, ( ...)) 
def convert_to_array(data):
    
    return (
        data[0],
        (
            data[1],
            data[2],
            np.array([float(data[3][0]), float(data[3][1]), float(data[3][2])]),  # from tuple to array
            np.array([float(data[4][0]), float(data[4][1]), float(data[4][2])]),  # from tuple to array
            np.array([float(data[5][0]), float(data[5][1]), float(data[5][2])]),  # from tuple to array
            float(data[6]) # to standard float
        )
    )

In [ ]:
# simulations parameter
sim_pars_file = np.loadtxt("/mnt/cosmo_GNN/latin_hypercube_params.txt", dtype=float)

# number of simulations to be processed
n_sims = 2000

# path list with simulation keys
path_list = [(i, "/mnt/cosmo_GNN/Data/" + str(i)) for i in range(n_sims)]

# parallelize path list and read files
fof_rdd = sc.parallelize(path_list)\
            .mapValues(read_cosmo_data)

# get positions and masses for each point
pos_mass_rdd = fof_rdd.mapValues(get_pos_mass)\
                      .flatMap(assign_key_to_rows)

# cut percentile
cut = 0.997

# get mass cuts 
mass_cut_rdd = fof_rdd.mapValues(get_pos_mass)\
                      .mapValues(lambda x: np.quantile(x[:, -1], cut))

In [ ]:
create_mass_hist(mass_cut_rdd, n_sims=n_sims)

In [ ]:
# collect mass cuts into numpy array
mass_cuts = mass_cut_rdd.values().collect()
mass_cuts = np.array(mass_cuts)

# filter by mass
pos_mass_rdd_filtered = pos_mass_rdd.filter(lambda x: x[1][-1] >= mass_cuts[x[0]])

# number of halos in each simulation
n_halos = pos_mass_rdd_filtered.countByKey()

# number of halos rdd
n_halos_rdd = sc.parallelize(n_halos.values())

In [ ]:
create_halos_hist(n_halos_rdd, n_sims=n_sims)

In [ ]:
# positions rdd ---> (simkey, pos)
pos_rdd = pos_mass_rdd_filtered.mapValues(lambda x: x[:3])

# create pairs
pairs_dist_rdd = create_pairs(pos_mass_rdd_filtered, divisions=4)

# filter by linking radius
linked_pairs_dist_rdd = pairs_dist_rdd.filter(lambda x: x[1][-1] <= 0.2)

# pairs rdd --> (simkey, (idx1, idx2)) --> reverse pair already included
pairs_rdd = linked_pairs_dist_rdd.mapValues(lambda x: (x[0], x[1]))

In [ ]:
# retrieve data for a simulation to show the plot
sim_num_plot = 8
pos_mass_rdd_filtered_plot = pos_mass_rdd_filtered.filter(lambda x: x[0] == sim_num_plot)\
                                                  .map(lambda x: x[1])
pos_mass_plot = np.array(pos_mass_rdd_filtered_plot.collect())
pos_plot = pos_mass_plot[:, :3]
mass_plot = pos_mass_plot[:, 3]
pairs_idx_plot = pairs_rdd.filter(lambda x: x[0] == sim_num_plot)\
                          .values()
pairs_idx_plot_array = np.array(pairs_idx_plot.collect())

In [ ]:
# edge attributes
edge_attr_rdd = get_edge_attributes(pos_rdd, linked_pairs_dist_rdd)

# group by simulation
grouped_idx_pos_rdd = pos_mass_rdd_filtered.groupByKey()\
                                 .mapValues(list)

grouped_edge_rdd = edge_attr_rdd.groupByKey()\
                                .mapValues(list)

# parallelize simulation parameters file and global features
param_rdd = sc.parallelize([(i, el) for i, el in enumerate(sim_pars_file)])
u = sc.parallelize([(i[0], math.log10(i[1])) for i in n_halos.items()])

# graph rdd (a graph for each simulation)
# masses, positions, simulation parameters, global features, edge indexes, edge features
raw_graph_rdd = grouped_idx_pos_rdd.join(grouped_edge_rdd)\
                                   .join(u)\
                                   .join(param_rdd)\
                                   .mapValues(lambda x: (x[0][0][0], x[0][0][1], x[0][1], x[1]))

graph_rdd = raw_graph_rdd.mapValues(lambda x: create_graph(x))   

In [ ]:
graph_rdd.collect()